# Data loading

## Speed test

Test the speed difference between a read from a raster image and a read from an hdf5 file

## raster images

In [2]:
import rasterio
from rich.console import Console
from rich.table import Table

from main.FolderInfos import FolderInfos
import os
import re
import numpy as np
import time


def get_array_raster_file(path):
    with rasterio.open(path) as file_object:
        dataset = file_object.read(1)
    return dataset,file_object

FolderInfos.init(test_without_data=True)
files = [FolderInfos.data_test+f for f in os.listdir(FolderInfos.data_test)]
dico_by_extensions = {}
for sf,f in zip(os.listdir(FolderInfos.data_test),files):
    name = re.sub("^([A-Za-z0-9_]+[A-Za-z_]{,2})(20[0-9_A-Z]+)(\\.[a-z]+)$","\\2",sf)
    ext = sf.split(".")[-1]
    if ext not in dico_by_extensions:
        dico_by_extensions[ext] = {}
    dico_by_extensions[ext][name] = f

# Measures...
list_imgs_ids = ["027481_0319CB_0EB7","016505_01F10F_CE84","016753_01F88A_4864"]
dico_times = {k:{"values":[],"shape":None,"mem_size":None} for k in list_imgs_ids}
for _ in range(50):
    for [[name,path],uniq_id] in zip(dico_by_extensions["img"].items(),list_imgs_ids):
        initial_time = time.time_ns()
        dataset, file_object = get_array_raster_file(path)
        dico_times[uniq_id]["values"].append(time.time_ns()-initial_time)
        dico_times[uniq_id]["shape"] = dataset.shape
        dico_times[uniq_id]["mem_size"] = dataset.nbytes
print("We have the following access time for each image:")
console = Console(color_system="windows")
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Name")
table.add_column("Access time (ms)")
table.add_column("Shape of the image")
table.add_column("Memory size (MB)")
for name_img,values in dico_times.items():
    table.add_row(
        f"{name_img} avg time", str(np.mean(values["values"])*1e-6),str(values["shape"]),str(values["mem_size"]*1e-6)
    )
all_times = [v["values"] for v in dico_times.values()]
table.add_row(
        f"Global avg time", str(np.mean(np.concatenate(all_times,axis=0)*1e-6)),"-"
    )
console.print(table)
        

We have the following access time for each image:


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Name                        ┃ Access time (ms)   ┃ Shape of the image ┃ Memory size (MB)  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ 027481_0319CB_0EB7 avg time │ 866.1529539999999  │ (10600, 18441)     │ 781.8983999999999 │
│ 016505_01F10F_CE84 avg time │ 971.112996         │ (10655, 18706)     │ 797.2497199999999 │
│ 016753_01F88A_4864 avg time │ 1025.8900019999999 │ (10648, 18759)     │ 798.9833279999999 │
│ Global avg time             │ 954.3853173333333  │ -                  │                   │
└─────────────────────────────┴────────────────────┴────────────────────┴───────────────────┘

In [ ]:
import h5py
images_hdf5 = h5py.File(f"{FolderInfos.input_data_folder}test_tmp_images.hdf5","w")

# Measures...
list_imgs_ids = ["027481_0319CB_0EB7","016505_01F10F_CE84","016753_01F88A_4864"]
dico_times = {k:{"valuesRead":[],"valuesWrite":[],"shape":None,"mem_size":None} for k in list_imgs_ids}
for [[name,path],uniq_id] in zip(dico_by_extensions["img"].items(),list_imgs_ids):
    ## Open the raster
    with rasterio.open(path) as raster: ## (NB: with keyword allows to manage files (properly open and close them)
        image_array: np.ndarray = raster.read(1) # Get the image array
        dico_times[uniq_id]["shape"] = image_array.shape
        dico_times[uniq_id]["mem_size"] = image_array.nbytes

    initial_time = time.time_ns()
    images_hdf5.create_dataset(name,shape=image_array.shape,dtype='f',
                                   data=image_array)
    dico_times[uniq_id]["valuesWrite"].append(time.time_ns()-initial_time)

images_hdf5.flush()
images_hdf5.close()
images_hdf5 = File(f"{FolderInfos.input_data_folder}test_tmp_images.hdf5","r")

for [[name,path],uniq_id] in zip(dico_by_extensions["img"].items(),list_imgs_ids):
    initial_time = time.time_ns()
    array = images_hdf5[uniq_id]
    dico_times[uniq_id]["valuesRead"].append(time.time_ns()-initial_time)
    
    
console = Console(color_system="windows")
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Name")
table.add_column("Access time write (ms)")
table.add_column("Access time read (ms)")
table.add_column("Shape of the image")
table.add_column("Memory size (MB)")
for name_img,values in dico_times.items():
    table.add_row(
        f"{name_img} avg time", str(np.mean(values["valuesWrite"])*1e-6), str(np.mean(values["valuesRead"])*1e-6),str(values["shape"]),str(values["mem_size"]*1e-6)
    )
all_times_read = [v["valuesRead"] for v in dico_times.values()]
all_times_write = [v["valuesWrite"] for v in dico_times.values()]
table.add_row(
        f"Global avg time", str(np.mean(np.concatenate(all_times_write,axis=0)*1e-6)),str(np.mean(np.concatenate(all_times_read,axis=0)*1e-6)),"-"
    )
console.print(table)    